# Notebook Context

This is 3 of 6 Jupyter Notebooks associated with the used car project.

This Notebook covers the GeoCoding aspect of the project which, in the end, was found to not improve model performance at all. How disappointing! The modelling results can be observed in notebook [06_Modelling_version2](https://github.com/rgdavies92/used-car-value/blob/main/06_Modelling_version2.ipynb).

# GeoCoding to obtain latitude and longitude from car listing information

In [1]:
# Import packages

import pandas as pd
import glob
import numpy as np
import regex
import re
from tqdm import tqdm
import urllib.parse
from urllib.request import urlopen
import json
import datetime 
import time
from bs4 import BeautifulSoup
import warnings
import pprint
import googlemaps
from geopy.geocoders import Nominatim
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Import cleaned cars data

abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/00_all_cars.csv'
df = pd.read_csv(abspath)
df.sample(5)

,name,name_subtitle,year,price,body,mileage,BHP,doors,transmission,make,...,seller1,used,e_engine_kWh,log_price,log_mileage,orig_name,id,year_reg,link,href0
191831,hyundai tucson,1.7 CRDi Blue Drive Premium 5dr 2WD Diesel Estate,2018,17000.0,SUV,43161.0,114.398422,5dr,Manual,Hyundai,...,4.7,1,NaN,9.740969,10.672693,Hyundai Tucson,202112010102988,2018 (18 reg),https://www.autotrader.co.uk/car-details/20211...,/dealers/midlothian/edinburgh/edinburgh-west-h...
129006,audi a5,3.0 TDI 245 Quattro S Line 2dr S Tronic [Nav],2016,20049.0,Coupe,18222.0,242.000000,2dr,Automatic,Audi,...,4.4,1,NaN,9.905935,9.810385,Audi A5,202201061069660,2016 (65 reg),https://www.autotrader.co.uk/car-details/20220...,/dealers/yorkshire/york/affordable-cars-presti...
153949,volkswagen polo,Volkswagen Polo 1.2 TSI 90 Match Edition 5dr DSG,2017,13599.0,Hatchback,26168.0,89.000000,5dr,Automatic,Volkswagen,...,4.9,1,NaN,9.517752,10.172293,Volkswagen Polo,202111249885913,2017 (67 reg),https://www.autotrader.co.uk/car-details/20211...,/dealers/staffordshire/cannock/available-car-s...
208607,vauxhall astra,Vauxhall Astra Estate 1.6 CDTi 110 Tech Line N...,2018,12299.0,Estate,39141.0,108.481262,5dr,Manual,Vauxhall,...,4.9,1,NaN,9.417273,10.574926,Vauxhall Astra,202201181466676,2018 (18 reg),https://www.autotrader.co.uk/car-details/20220...,/dealers/staffordshire/cannock/available-car-s...
211958,volkswagen golf,1.4 TSI S 5dr,2018,14490.0,Hatchback,31004.0,123.274162,5dr,Manual,Volkswagen,...,4.7,1,NaN,9.581214,10.341872,Volkswagen Golf,202112160564574,2018 (18 reg),https://www.autotrader.co.uk/car-details/20211...,/dealers/neath-port-talbot/port-talbot/sinclai...


In [3]:
# Define function which uses nominamtim to return lat/lon for each car area

# This function takes a low (l) and high (h) index as parameters for which to return latitude and longitude
# It takes a VERY long time, at approximately 2s per car. I would recommend running it in swaths.
# The function will output a csv containing car.name, car.id, car.dealer_city, car.dealet_lat and car.dealer_lon
# which can be merged with the cleaned car data. This process is done in Notebook 04.
def get_latlon_to_csv_used(l, h):
    lat_series = []
    lon_series = []
    # dealer city holder
    dc = []
    # car ID holder
    cid = []
    name = []
    # subset the car dataframe based on the index of rows that you want to obtain lat lon for
    ldf = df.loc[l:h,['name','id','dealer_city']]

    for index, car in ldf.iterrows():
        try:
            address = car[-1] + ' UK'
            # Indert the dealer_city and 'UK' into the nominatim search
            url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
            response = requests.get(url).json()
            # Extract the relevant lat/lon data from the json response.
            lat_series.append(' '+str(response[0]["lat"]))
            lon_series.append(' '+str(response[0]["lon"]))
            # Append corresponding car details for merge later
            cid.append(car[-2])
            dc.append(car[-1])
            name.append(car[-3])
        except:
            pass

    # throw complete entries into a dataframe and output to csv.     
    odf = pd.DataFrame({'name': name,
                       'id': cid,
                       'dealer_city':dc,
                       'dealer_lat':lat_series,
                        'dealer_lon':lon_series})    
    now = (datetime.datetime.now().strftime("%d%B_%I%M%p"))
    filepathdf=f'df_latlon/{l}_{h}_{now}_latlon_used.csv'
    odf.to_csv(filepathdf, index=False, header=odf.columns )   
    # Function also returns the dataframe
    return odf

In [4]:
# Call function for a managable subset of the dataset. Managable is situation dependent! 

# In hindsight I might have tried to run this for unique dealer_city entries only and then 
# did a one-many merge afterwards rather than this computationally expensive way. Fortunately 
# time wasn't an issue for me. 

# df_example = get_latlon_to_csv_used(0,10)

# GeoCoding to obtain Dealer County from latitude and longitude

In [5]:
# Load the accumulated car data
abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/00_all_cars_ll.csv'
df = pd.read_csv(abspath)

In [6]:
df.columns

Index(['name', 'name_subtitle', 'year', 'price', 'body', 'mileage', 'BHP',
       'doors', 'transmission', 'make', 'fuel', 'mpg', 'drivertrain', 'engine',
       'owners', 'ULEZ', 'county', 'dealer_area', 'dealer_city', 'dealer_lat',
       'dealer_lon', 'geocode', 'county.1', 'postcode', 'postcode_short',
       'postcode_shortest', 'seller1', 'used', 'e_engine_kWh', 'log_price',
       'log_mileage', 'orig_name', 'id', 'year_reg', 'link', 'href0'],
      dtype='object')

In [7]:
# Iterate through the latitude and longitude columns to extract unique occurences
dealer_latlon = []
for index, car in df.iterrows():
    latlon =  [car[-17],car[-16]]
    if latlon not in dealer_latlon:
        dealer_latlon.append(latlon)
len(dealer_latlon)

3270

In [8]:
len(df.dealer_lat.value_counts())

1720

In [9]:
dealer_latlon[:5]

[[52.688227, -2.0324197],
 [52.3679896, -2.7193114],
 [53.92023225, -1.824272287202847],
 [52.5847949, -1.9822687],
 [50.9025349, -1.404189]]

In [10]:
abspath = r'/Users/robertdavies/Desktop/DSI/GA_P2/LargeDatasets/counties.csv'
counties = pd.read_csv(abspath)

In [15]:
# Define my geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Make holding lists for the original lat/lon and the returned geocode
geocode = []
original_ll = []

# Loop over all 3740 unique lat/lon locations
for ll in tqdm(dealer_latlon):
    try:
        geocode.append(geolocator.reverse(str(ll[0])+","+str(ll[1])))
        original_ll.append(ll)
    except:
        geocode.append('bad result')
        original_ll.append(ll)

100%|███████████████████████████████████████████| 3270/3270 [14:19<00:00,  3.80it/s]


In [16]:
print(len(dealer_latlon))
print(len(original_ll))
print(len(geocode))

3270
3270
3270


In [17]:
df_base = pd.DataFrame({'base_ll':dealer_latlon,'original_ll':original_ll,'geocode':geocode})

In [18]:
df_base.sample(5)

,base_ll,original_ll,geocode
2318,"[nan, nan]","[nan, nan]",bad result
2261,"[nan, nan]","[nan, nan]",bad result
2223,"[nan, nan]","[nan, nan]",bad result
775,"[53.6975234, -2.2004698]","[53.6975234, -2.2004698]","(Bacup, Manor Street, Rockliffe, Bacup, Rossen..."
2311,"[nan, nan]","[nan, nan]",bad result


In [19]:
# Count the number of bad results
print('Number of bad results is:',df_base[df_base.geocode=='bad result'].shape)

# Drop all bad results, these won't be useful
df_base = df_base[df_base.geocode!='bad result'].copy()

Number of bad results is: (1550, 3)


In [20]:
# Add county column with all results as 'bad_Result'
df_base['county']= 'bad result'

# Iterate over rows in df_base and check for one of the counties in the address holder
# If it's in there, then assign the county to df_base.county
for index, loc in df_base.iterrows():
    address = str(loc[-2])
    for county in counties.County:
        if county in address:
            df_base.loc[index,'county'] = county
                     

In [21]:
df_base.sample(5)

,base_ll,original_ll,geocode,county
266,"[52.2274533, 0.4853366]","[52.2274533, 0.4853366]","(Chapel Row, Church Street, Ashley, East Cambr...",Cambridgeshire
1450,"[50.526735200000005, -3.755984991182407]","[50.526735200000005, -3.755984991182407]","(Ashburton, Teignbridge, Devon, South West Eng...",Devon
687,"[55.069836, -3.6092292]","[55.069836, -3.6092292]","(Fleshers, Loreburn Street, Troqueer, Dumfries...",Dumfries and Galloway
388,"[51.9925394, -2.156016959290844]","[51.9925394, -2.156016959290844]","(71-73, Barton Street, Priors Park, Tewkesbury...",Gloucestershire
1168,"[51.7299485, -3.1359867]","[51.7299485, -3.1359867]","(Abertillery Library, Castle Street, Abertille...",bad result


In [22]:
# Add postcode_short column with all results as 'bad_Result'
df_base['postcode']= 'bad result'
df_base['postcode_short']= 'bad result'

# Iterate over rows in df_base and use RegEx to extract postcodes from the address where possible
for index, loc in df_base.iterrows():
    address = str(loc[-4])
    try:
        po = re.findall('[A-Z][A-Z]?[0-9][0-9]?[ ]?[0-9][A-Z][A-Z]', address)[0]
        df_base.loc[index,'postcode'] = po
    except:
        pass

# Use a lambda function to pinch off just the start of the postcode    
df_base['postcode_short']=df_base.postcode.apply(lambda x: (x[:-3]).strip())

In [23]:
df_base.sample(5)

,base_ll,original_ll,geocode,county,postcode,postcode_short
1833,"[52.0735984, -4.1548796]","[52.0735984, -4.1548796]","(Llanybydder, Sir Gaerfyrddin / Carmarthenshir...",Carmarthenshire,SA40 9RX,SA40
1213,"[51.5285171, 0.4772824]","[51.5285171, 0.4772824]","(Wharf Road, Fobbing, Thurrock, East of Englan...",bad result,SS17 9JN,SS17
674,"[51.6441535, -0.3622403]","[51.6441535, -0.3622403]","(Bushey Country Club, High Street, Bushey, Bus...",Hertfordshire,WD23 1TT,WD23
18,"[51.2715316, -0.3414523511290909]","[51.2715316, -0.3414523511290909]","(Norbury Park House, Druids Grove, Mickleham, ...",Surrey,RH5 6ER,RH5
635,"[55.8455828, -4.4239646]","[55.8455828, -4.4239646]","(Paisley Cenotaph, Dyer's Wynd, Castlehead, Pa...",Renfrewshire,PA1 1BG,PA1


In [24]:
df_base.postcode_short.value_counts()

bad res    114
LE9          4
OX5          4
SA6          3
CO13         3
          ... 
TF9          1
NR19         1
WS7          1
EN10         1
GL17         1
Name: postcode_short, Length: 1405, dtype: int64

Good so far - there are only 114 areas that I haven't been able to get a postcode for. 

Now I'll try a different method to get some values for them.

In [25]:
# Make a new dataframe, just of the bas results
df_base_bad = df_base[df_base.postcode_short=='bad res'].copy()

In [26]:
df_base_bad.sample(5)

,base_ll,original_ll,geocode,county,postcode,postcode_short
833,"[50.84279555, -3.3689405321011514]","[50.84279555, -3.3689405321011514]","(Cullompton, Mid Devon, Devon, South West Engl...",Devon,bad result,bad res
1550,"[52.4035742, -1.8451962]","[52.4035742, -1.8451962]","(Shirley, Haslucks Green Road, Haslucks Green,...",West Midlands,bad result,bad res
557,"[55.7743458, -3.9184134]","[55.7743458, -3.9184134]","(Wishaw (N Lanarks), before Stewarton Street, ...",North Lanarkshire,bad result,bad res
453,"[51.0107646, -4.199478523283679]","[51.0107646, -4.199478523283679]","(Kynock Industrial Estate, Bideford, Torridge,...",Devon,bad result,bad res
1361,"[53.3650134, -1.5432431]","[53.3650134, -1.5432431]","(Christ Church Fulwood, Canterbury Avenue, Ful...",bad result,bad result,bad res


In [27]:
# This time I use a getplace function which makes a call to the Google maps API. I've had to remove my 
# Key so it's not in the project. You can make your own for free.
def getpostcode(lat, lon):
    # Function takes lat and lon as input parameters and uses them and my API key to craft the url
    url = "https://maps.googleapis.com/maps/api/geocode/json?key=YOURKEYHERE"
    url += "&latlng=%s,%s&sensor=false" % (lat, lon)
    v = urlopen(url).read()
    # Load the returned json
    j = json.loads(v)
    # The json returns many different options. First I'll look through the first element for a postcode
    components = j['results'][0]['address_components']
    # Initialise empty results. Use bad result again for postcode
    country = town = county = None
    postcode = 'bad result'
    for c in components:
        if 'administrative_area_level_2' in c['types']:
            county = c['long_name']
        if "country" in c['types']:
            country = c['long_name']
        if "postal_code" in c['types']:
            postcode = c['long_name']
        if "postal_town" in c['types']:
            town = c['long_name']

    # If the postcode is still 'bad result' then the first elementof json didn't contain a postcode. 
    # Try second element of json.   
    if postcode == 'bad result':
        components = j['results'][1]['address_components']
        for c in components:
            if 'administrative_area_level_2' in c['types']:
                county = c['long_name']
            if "country" in c['types']:
                country = c['long_name']
            if "postal_code" in c['types']:
                postcode = c['long_name']
    
    # I've muted this function to return the postcode only as that's all I'm interested in at this stage.
    return postcode #, town, county, country

In [28]:
df_base_bad['postcode']='bad result'
# Iterate over rows in the bad postcode dataframe. For each row, run the getpostcode function and use .loc
# to assign the returned postcode to the df_base dataframe
for index, row in df_base_bad.iterrows():
    lat = row[1][0]
    lon = row[1][1]
    try:
        df_base.loc[index,'postcode'] = getpostcode(lat, lon)
    except:
        df_base.loc[index,'postcode'] = ' bad result'
        
df_base['postcode_short']=df_base.postcode.apply(lambda x: (x[:-3]).strip())

In [29]:
df_base.sample(5)

,base_ll,original_ll,geocode,county,postcode,postcode_short
143,"[54.8518781, -1.8333741]","[54.8518781, -1.8333741]","(Eyesite - Finlays, Middle Street, Consett, Co...",Durham,DH8 5AB,DH8
380,"[51.5754602, 0.4757363]","[51.5754602, 0.4757363]","(Bonnygate, Basildon, Essex, East of England, ...",Essex,SS14 2QN,SS14
1071,"[51.713353, -3.445555]","[51.713353, -3.445555]","(St Elvan's, Church Street, Gadlys, Aberdare E...",bad result,CF44 7AB,CF44
1854,"[52.6133558, -2.4827324]","[52.6133558, -2.4827324]","(Broseley House, High Street, Broseley, Shrops...",West Midlands,TF12 5EZ,TF12
1604,"[51.5910395, -0.1420768]","[51.5910395, -0.1420768]","(Bus Depot, A504, Muswell Hill, London Borough...",Greater London,N10 1DF,N10


In [30]:
df_base.postcode.value_counts()

WS11 1AE    1
KY1 4RR     1
SL6 2JP     1
EH26 8HN    1
NP23 6ET    1
           ..
SW8 2JL     1
GL13 9BP    1
ST15 8AD    1
BA11 1PU    1
GL17 9SE    1
Name: postcode, Length: 1720, dtype: int64

Great, no more bad results in the postcode column. But there are in the county column. Now I go through the same procedure for them.

In [31]:
df_base.county.value_counts()

bad result          218
Greater London      187
West Midlands        80
Surrey               62
Kent                 53
                   ... 
East Ayrshire         2
Orkney Islands        1
Glasgow City          1
Fermanagh             1
Shetland Islands      1
Name: county, Length: 79, dtype: int64

In [32]:
# Make a new dataframe just of the bad results
df_base_bad2 = df_base[df_base.county=='bad result'].copy()

def getcounty(lat, lon):
    # Function takes lat and lon as input parameters and uses them and my API key to craft the url
    url = "https://maps.googleapis.com/maps/api/geocode/json?key=YOURKEYHERE"
    url += "&latlng=%s,%s&sensor=false" % (lat, lon)
    v = urlopen(url).read()
    # Load the returned json
    j = json.loads(v)
    # The json returns many different options. First I'll look through the first element for a postcode
    components = j['results'][0]['address_components']
    # Use bad result again for county
    county = 'bad result'
    for c in components:
        if 'administrative_area_level_2' in c['types']:
            county = c['long_name']

    # If the county is still 'bad result' then the first element of json didn't contain a postcode. 
    # Try second element of json.   
    if county == 'bad result':
        components = j['results'][1]['address_components']
        for c in components:
            if 'administrative_area_level_2' in c['types']:
                county = c['long_name']
    return county 

In [33]:
# Iterate over rows in the bad county dataframe. For each row, run the getcounty function and use .loc
# to assign the returned county to the df_base dataframe

for index, row in df_base_bad2.iterrows():
    lat = row[1][0]
    lon = row[1][1]
    try:
        df_base.loc[index,'county'] = getcounty(lat, lon)
    except:
        df_base.loc[index,'county'] = ' bad result'

In [34]:
df_base.sample(5)

,base_ll,original_ll,geocode,county,postcode,postcode_short
1694,"[53.6905487, -1.8394265]","[53.6905487, -1.8394265]","(Elland, Exley Lane, Elland, Calderdale, West ...",West Yorkshire,HX5 9HP,HX5
1847,"[51.46773895, -0.4587800741571181]","[51.46773895, -0.4587800741571181]","(London Heathrow Airport, Stanhope Heath, Stan...",Surrey,TW19 7PL,TW19
3013,"[52.9325093, -1.1897308455681412]","[52.9325093, -1.1897308455681412]","(Beeston Sidings Nature Reserve, Dr Stewart Ad...",Nottinghamshire,NG90 1BS,NG90
1601,"[55.8949497, -3.4642933]","[55.8949497, -3.4642933]","(230, Main Street, Calderwood, East Calder, We...",West Lothian,EH53 0EW,EH53
264,"[50.7651261, -1.8145483592492209]","[50.7651261, -1.8145483592492209]","(Christchurch Road, Hurn, Bournemouth, Christc...",Dorset,BH23 6AE,BH23


In [35]:
df_base.county.value_counts()

Greater London      187
West Midlands        80
Surrey               62
Kent                 53
Hertfordshire        43
                   ... 
Middlesbrough         1
bad result            1
Newport               1
Rutland               1
Shetland Islands      1
Name: county, Length: 136, dtype: int64

Excellent, no more null counties. All good to write out to a csv for merging in the data cleaning notebook.

In [36]:
# Sabve with a more sensible name and add some rounded lat/lon to make merging easy.
merge_df = df_base[['original_ll', 'geocode', 'county', 'postcode', 'postcode_short']].copy()
merge_df['dealer_lat']=merge_df.original_ll.apply(lambda x: x[0])
merge_df['dealer_lon']=merge_df.original_ll.apply(lambda x: x[1])
merge_df['dealer_lat_rnd']=merge_df.original_ll.apply(lambda x: round(x[0],5))
merge_df['dealer_lon_rnd']=merge_df.original_ll.apply(lambda x: round(x[1],5))

In [37]:
merge_df.columns

Index(['original_ll', 'geocode', 'county', 'postcode', 'postcode_short',
       'dealer_lat', 'dealer_lon', 'dealer_lat_rnd', 'dealer_lon_rnd'],
      dtype='object')

In [38]:
county_abspath = r'data/04_merge_counties.csv'
# merge_df.to_csv(county_abspath, index=False, header=merge_df.columns )

In [39]:
county_abspath = r'data/04_merge_counties.csv'
# merge_df = pd.read_csv(county_abspath)

In [40]:
merge_df.sample(10)

,original_ll,geocode,county,postcode,postcode_short,dealer_lat,dealer_lon,dealer_lat_rnd,dealer_lon_rnd
1869,"[55.8832144, -3.1147415]","(High Street, Lasswade, Midlothian, Alba / Sco...",Midlothian,EH18 1ND,EH18,55.883214,-3.114742,55.88321,-3.11474
820,"[52.9889575, -2.2026103]","(Flash Lane, Trent Vale, Stoke, Stoke-on-Trent...",West Midlands,ST4 5QZ,ST4,52.988957,-2.202610,52.98896,-2.20261
54,"[51.4058006, -0.1640787]","(Mitcham Fair Green, London Road, Mitcham, Lon...",Greater London,CR4 2JD,CR4,51.405801,-0.164079,51.40580,-0.16408
972,"[51.4625524, -0.2167462]","(Robert Dyas, 126,128, Putney High Street, Put...",Greater London,SW15 1RG,SW15,51.462552,-0.216746,51.46255,-0.21675
1474,"[52.0357457, -2.4287821]","(Ashmead, Woodleigh Road, Ledbury, Herefordshi...",West Midlands,HR8 2BL,HR8,52.035746,-2.428782,52.03575,-2.42878
282,"[52.6040015, 0.3809631901097164]","(Bexwell Road, Downham Market, King's Lynn and...",Down,PE38 9LH,PE38,52.604002,0.380963,52.60400,0.38096
2247,"[51.4795563, -3.7040704]","(Victoria Avenue, Porthcawl, Bridgend, Cymru /...",Bridgend County Borough,CF36 3HG,CF36,51.479556,-3.704070,51.47956,-3.70407
95,"[51.8784385, -0.4152837]","(NatWest, George Street, Park Town, Luton, Eas...",Luton,LU1 2AQ,LU1,51.878439,-0.415284,51.87844,-0.41528
885,"[53.1394364, -4.2769751]","(Castell Caernarfon, Stryd y Jêl / Shirehall...",Gwynedd,LL55 2AY,LL55,53.139436,-4.276975,53.13944,-4.27698
147,"[53.3645138, -3.0507757]","(Prenton Dell Road, Prenton, Wirral, North Wes...",Merseyside,CH43 3BT,CH43,53.364514,-3.050776,53.36451,-3.05078


There's no more content in this Notebook 3. Geocoding has been used to create dataframes for merging in with the AutoTrader data.